# Setting up the environment

In [ ]:
# Import and/or install libraries

import subprocess, os

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

In [ ]:
# Connect to Google Cloud

from google.cloud import storage
client = storage.Client()

# Getting started

We will start by using Google Dynamic World and using the built layer. It seems like we should also filter the certainty to be greater than 0.06 (highest false positive in the grassland had a certainty of 0.0569).

In [ ]:
# AOI

aoi = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              31.092023309844734,
              -24.821258854952376
            ],
            [
              31.092023309844734,
              -25.303992405280965
            ],
            [
              31.791037410539502,
              -25.303992405280965
            ],
            [
              31.791037410539502,
              -24.821258854952376
            ],
            [
              31.092023309844734,
              -24.821258854952376
            ]
          ]
        ],
      })


rock = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              31.284,
              -25.360
            ],
            [
              31.285,
              -25.360
            ],
            [
              31.285,
              -25.361
            ],
            [
              31.284,
              -25.361
            ],
            [
              31.284,
              -25.360
            ],
          ]
        ],
      })


town = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              31.14157858391627,
              -25.17820869636482
            ],
            [
              31.14157858391627,
              -25.180385520009708
            ],
            [
              31.143983964831108,
              -25.180385520009708
            ],
            [
              31.143983964831108,
              -25.17820869636482
            ],
            [
              31.14157858391627,
              -25.17820869636482
            ]
          ]
        ],
      })

# Prepare layers

First, we experiment with 2022 for our AOI.

In [ ]:
year = 2022
start_date = '{}-01-01'.format(year)
end_date = '{}-01-01'.format(year + 1)

# Google Dynamic World
people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('label').eq(6).selfMask()
certainty_mask = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').gt(0.06).selfMask()
best_people = people.mask(certainty_mask).eq(1).selfMask()

# Google Open Buildings
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v2/polygons').filter('confidence >= 0.70');

buildings_raster = buildings.reduceToImage(
  properties=['confidence'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

# Join with other layers
built = best_people.unmask(0).add(buildings_raster.unmask(0)).gt(0).selfMask()

Rasterise Google's Open Buildings

# SAR for rocks

In [ ]:
ffa_db = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(ee.Date('2022-01-01'), ee.Date('2023-01-01')).filterBounds(aoi).mean()
ffa_fl = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(ee.Date('2022-01-01'), ee.Date('2023-01-01')).filterBounds(aoi).mean()

# Add VV-VH ratio
def add_ratio(image):
    ratio = image.select('VV').divide(image.select('VH')).rename('VV_VH_ratio')
    return image.addBands(ratio)

# Load Sentinel-1 data
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                .filterBounds(aoi)
                .filterDate('2022-01-01', '2023-01-01')
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.eq('instrumentMode', 'IW'))).map(add_ratio)


image = collection.mean()




# Import training data

In [ ]:
import json

filename = "/content/drive/MyDrive/mygit/naturewatch_analysis/geometries/rocks.geojson"

# Load a GeoJSON file
with open(filename, 'r') as f:
    data = json.load(f)

# Convert the GeoJSON into an ee.FeatureCollection
fc = ee.FeatureCollection(data['features'])


In [ ]:
num_samples = 500
classes = [0, 1]
sample_list = []

def addAttribute(feature, value):
    return feature.set('label', value)

for cls in classes:
  polygons_class = fc.filter(ee.Filter.eq('label', cls))
  randomPoints = ee.FeatureCollection.randomPoints(polygons_class, num_samples, 0, 1);
  randomPoints = randomPoints.map(lambda feature: addAttribute(feature, cls))
  sample_list.append(randomPoints)

samples_all = sample_list[0].merge(sample_list[1])


In [ ]:
# Select features for classification
inputFeatures = ['VV', 'VH', 'VV_VH_ratio']

# Extract band values for each training region
samples = image.select(inputFeatures).sampleRegions(collection=samples_all, properties=['label'], scale=10)

# Train the classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50).train(features=samples, classProperty='label', inputProperties=inputFeatures)

# Classify the images
result = image.classify(classifier)

# Summary statistics

In [ ]:
# Separate samples for each class
samples_0 = samples.filter(ee.Filter.eq('label', 0))
samples_1 = samples.filter(ee.Filter.eq('label', 1))

band_oi = 'VV'

# Min of each class
mins_0 = samples_0.reduceColumns(reducer=ee.Reducer.min(), selectors=[band_oi])
print('Min class 0: ', mins_0.getInfo())

mins_1 = samples_1.reduceColumns(reducer=ee.Reducer.min(), selectors=[band_oi])
print('Min class 1: ', mins_1.getInfo())

# Max of each class
max_0 = samples_0.reduceColumns(reducer=ee.Reducer.max(), selectors=[band_oi])
print('Max class 0: ', max_0.getInfo())

max_1 = samples_1.reduceColumns(reducer=ee.Reducer.max(), selectors=[band_oi])
print('Max class 1: ', max_1.getInfo())

# Mean of each class
means_0 = samples_0.reduceColumns(reducer=ee.Reducer.mean(), selectors=[band_oi])
print('Means class 0: ', means_0.getInfo())

means_1 = samples_1.reduceColumns(reducer=ee.Reducer.mean(), selectors=[band_oi])
print('Means class 1: ', means_1.getInfo())

# Standard deviation of each class
std_devs_0 = samples_0.reduceColumns(reducer=ee.Reducer.stdDev(), selectors=[band_oi])
print('Standard deviations class 0: ', std_devs_0.getInfo())

std_devs_1 = samples_1.reduceColumns(reducer=ee.Reducer.stdDev(), selectors=[band_oi])
print('Standard deviations class 1: ', std_devs_1.getInfo())


In [ ]:
# Exctract bands
samples_info = samples.getInfo()
VV_values = [feat['properties']['VV'] for feat in samples_info['features']]
VH_values = [feat['properties']['VH'] for feat in samples_info['features']]
VV_VH_ratio = [feat['properties']['VV_VH_ratio'] for feat in samples_info['features']]


labels = [feat['properties']['label'] for feat in samples_info['features']]

# Convert to pandas dataframe
import pandas as pd
df = pd.DataFrame({
    'VV': VV_values,
    'VH': VH_values,
    'VV_VH_ratio': VV_VH_ratio,
    'label': labels
})

df_0 = df[df['label'] == 0]
df_1 = df[df['label'] == 1]

# Make histograms
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.subplot(131)
plt.hist(df_0['VV'], bins=50, alpha=0.5, label='class 0')
plt.hist(df_1['VV'], bins=50, alpha=0.5, label='class 1')
plt.title('VV distribution')
plt.legend()

plt.subplot(132)
plt.hist(df_0['VH'], bins=50, alpha=0.5, label='class 0')
plt.hist(df_1['VH'], bins=50, alpha=0.5, label='class 1')
plt.title('VH distribution')
plt.legend()

plt.subplot(133)
plt.hist(df_0['VV_VH_ratio'], bins=50, alpha=0.5, label='class 0')
plt.hist(df_1['VV_VH_ratio'], bins=50, alpha=0.5, label='class 1')
plt.title('VV_VH_ratio')
plt.legend()

plt.show()

print(min(df_1['VV_VH_ratio']))


# Make polygons of built

In [ ]:
# built_aoi = best_people.clipToCollection(aoi);

# Convert to polygons
built_polygons = best_people.reduceToVectors(
  geometry=aoi,
  crs=best_people.projection(),
  scale=20,
  geometryType='polygon',
  eightConnected=False,
  labelProperty='label',
  maxPixels=10453920,
);

# Sample max VV and VH values
poly_max =image.select(['VV', 'VH']).reduceRegions(
    collection=built_polygons,
    reducer=ee.Reducer.max(),
    scale=10)

img_max_VV = poly_max.reduceToImage(
    properties = ['VV'],
    reducer = ee.Reducer.first()
).rename('VV')

img_max_VH = poly_max.reduceToImage(
    properties = ['VH'],
    reducer = ee.Reducer.first()
).rename('VH')

img_max = img_max_VV.addBands(img_max_VH)

# vv_rocks = vv_max.filter(ee.Filter.lt('max', -8))

# Build a random forest model
Making rasters of the polygons again, but assigning each pixel its polygon value and then building a classifier to predict the new rasters.

In [ ]:
# Select features for classification
inputFeatures = ['VV', 'VH']

# Extract band values for each training region
samples = img_max.select(inputFeatures).sampleRegions(collection=samples_all, properties=['label'], scale=10)

# Train the classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50).train(features=samples, classProperty='label', inputProperties=inputFeatures)

# Classify the images
result = img_max.classify(classifier)

# Map

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')

# Map.addLayer(people, {'min':0, 'max':1, 'palette':['white','blue']}, 'people')
# Map.addLayer(best_people, {'min':0, 'max':1, 'palette':['white','red']}, 'best_people')

# Map.addLayer(buildings, {'color': 'red'}, 'Buildings confidence >= 0.70');
# Map.addLayer(buildings_raster, {'min':0, 'max':1, 'palette':['white','red']}, 'buildings_raster')
# Map.addLayer(gdw, {'min':0, 'max':8, 'palette':['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b', 'a59b8f', 'b39fe1']}, 'GDW')
# Map.addLayer(people_c, {'min':0, 'max':0.1, 'palette':['white','blue']}, 'people_c')

# Map.addLayer(built, {}, 'built')

# Map.addLayer(ffa_db.select('VV'), {'min': -20, 'max': 0}, 'VV')
# Map.addLayer(ffa_db.select('VH'), {'min': -20, 'max': 0}, 'VH')

# Map.addLayer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'rgb')
# Map.addLayer(result, {}, 'result')
# Map.addLayer(samples, {}, 'fc')

Map.addLayer(img_max_VV, {'min': -14, 'max': -8, 'palette':['blue', 'white', 'red']}, 'vv_rocks') #-11
# Map.addLayer(img_max_VH, {'min': -18, 'max': -12, 'palette':['blue', 'white', 'red']}, 'vh_rocks') #-15

# Map.addLayer(bedrock, {}, 'bedrock')

Map.addLayer(result, {'min':0, 'max':1, 'palette':['white','blue']}, 'result')

Map.setCenter(31.273, -25.355, 16)
Map
